In [ ]:
from datasets import load_dataset

# Load the "train" split of the dataset
dataset = load_dataset("brunokreiner/genius-lyrics", split="train")


# Filter dataset to keep only English lyrics
filtered_dataset = dataset.filter(lambda example: example["is_english"])

# Keep only the "lyrics" column
lyrics_only = filtered_dataset.remove_columns(
    [col for col in filtered_dataset.column_names if col != "lyrics"]
)

# Create train and validation splits
train_val_split = lyrics_only.train_test_split(test_size=0.1, seed=42)  # 10% validation
train_set = train_val_split["train"]
val_set = train_val_split["test"]

print(f"Train size: {len(train_set)}, Validation size: {len(val_set)}")

In [2]:
# From the lyrics column, get only the text
train_set = train_set['lyrics']
val_set = val_set['lyrics']

In [ ]:
print(train_set[0:2])

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import re

In [ ]:
# Define hyperparameters
batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.mps.is_available() else 'cpu'
eval_iters = 200
n_embd = 10
dropout = 0.0
# ------------
# Adjusted hyperparameters for the second and third versions
# Second version: batch_size = 16, block_size = 32, max_iters = 7000, n_embd = 32
# Third version: batch_size = 32, block_size = 64, max_iters = 10000, n_embd = 64
# ------------
# Manual seed for reproducibility
torch.manual_seed(1337)

In [9]:
# Vocabulary and dictionaries
# Get all the unique characters that occur in this text
# Function to remove non-Latin characters and unwanted symbols
def filter_latin_chars(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

# Apply filtering to all the lyrics
filtered_train_set = [filter_latin_chars(lyric) for lyric in train_set]
filtered_val_set = [filter_latin_chars(lyric) for lyric in val_set]

# Remove empty lyrics after filtering (if any)
filtered_train_set = [lyric for lyric in filtered_train_set if lyric.strip()]
filtered_val_set = [lyric for lyric in filtered_val_set if lyric.strip()]


# Recreate the text from the filtered lyrics
train_text = "".join(filtered_train_set)
val_text = "".join(filtered_val_set)

chars = sorted(list(set((train_text))))
vocab_size = len(chars)
# Create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


In [ ]:
print("".join(chars))
print(vocab_size)

In [ ]:

# Split the dataset
n = int(0.9 * len(train_text))  # 90% for training
train_text = train_text[:n]
val_text = train_text[n:]

# Encode text into integer indices
train_set = encode(train_text)
val_set = encode(val_text)

# Convert lists to PyTorch tensors
train_set = torch.tensor(train_set, dtype=torch.long)
val_set = torch.tensor(val_set, dtype=torch.long)

print(f"Train size: {len(train_set)}, Validation size: {len(val_set)}")



In [ ]:
# Data loading
# Adapt the Andrej's code to the new training data
def get_batch(split):
    # Select dataset
    data = train_set if split == 'train' else val_set
    # Sample random starting indices for batch
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Directly slice the tensor (no need to encode)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # Move to device
    x, y = x.to(device), y.to(device)
    return x, y

# Test the function
print(get_batch('train'))



In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            #print('estimate loss',X.shape, Y.shape)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out # (B, T, hs)

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd) # apply one head of self-attention. (B, T, hs=C)
        self.lm_head = nn.Linear(n_embd, vocab_size) # (C, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.sa_head(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # (B,T,C)
            logits = logits.view(B*T, C) # (B*T, C)
            targets = targets.view(B*T) # (B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = BigramLanguageModel()
m = model.to(device)
# Print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
# Training
for iter in range(max_iters):

    # Every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
# Generate the lyrics
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(m.generate(context, max_new_tokens=1000)[0].tolist())
print(generated_text)